This currently does not function, in the middle of a rewrite

In [15]:
# Import PyTorch
import torch
# Get pandas for data manipulation
import pandas as pd
# Import nltk for text processing
import nltk
# Import os for file manipulation
import os

In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("Using CUDA")
else:
    print("CUDA is not available. Using CPU")

Using CUDA


In [3]:
# Load the data from Datasets/onehotplotgenre.csv
data = pd.read_csv("Datasets/onehotplotgenre.csv")
# Get the first 5 rows of the data
print(data.head())

                                    Title  \
0                             # (2012/II)   
1          #1 Cheerleader Camp (2010) (V)   
2                 #1 Serial Killer (2013)   
3  #1 at the Apocalypse Box Office (2015)   
4                             #137 (2011)   

                                                Plot  Action  Adventure  \
0  The night falls on the big city and a hooded f...       0          0   
1  When they're hired to work at a cheerleading c...       0          0   
2  Years of seething rage against the racism he's...       0          0   
3  Jules is, self declared, the most useless pers...       0          0   
4  #137 is a SCI/FI thriller about a girl, Marla,...       0          0   

   Animation  Biography  Comedy  Crime  Drama  Family  ...  Horror  Music  \
0          1          0       0      0      0       0  ...       0      0   
1          0          0       1      0      0       0  ...       0      0   
2          0          0       0      0      0  

In [4]:
# Create a dataframe with the plot column tokenized and lowercased
tokenizeddf = data.copy()
tokenizeddf['Plot'] = tokenizeddf['Plot'].apply(lambda x: nltk.word_tokenize(x.lower()))

In [5]:
print(tokenizeddf.head())

                                    Title  \
0                             # (2012/II)   
1          #1 Cheerleader Camp (2010) (V)   
2                 #1 Serial Killer (2013)   
3  #1 at the Apocalypse Box Office (2015)   
4                             #137 (2011)   

                                                Plot  Action  Adventure  \
0  [the, night, falls, on, the, big, city, and, a...       0          0   
1  [when, they, 're, hired, to, work, at, a, chee...       0          0   
2  [years, of, seething, rage, against, the, raci...       0          0   
3  [jules, is, ,, self, declared, ,, the, most, u...       0          0   
4  [#, 137, is, a, sci/fi, thriller, about, a, gi...       0          0   

   Animation  Biography  Comedy  Crime  Drama  Family  ...  Horror  Music  \
0          1          0       0      0      0       0  ...       0      0   
1          0          0       1      0      0       0  ...       0      0   
2          0          0       0      0      0  

In [6]:
# Get the length of the longest plot
maxlen = tokenizeddf['Plot'].apply(len).max()

# Get the set of all words in the plot column
wordset = set()
for plot in tokenizeddf['Plot']:
    wordset.update(plot)
# Get the number of unique words
numwords = len(wordset)
print("Number of unique words: ", numwords)

# Create a dictionary that maps words to integers
word2int = {word: i for i, word in enumerate(wordset)}

# Function to convert a list of words to a list of integers
def words2ints(words):
    return [word2int[word] for word in words]

Number of unique words:  520798


In [7]:
# Convert the plot column to a list of integers
tokenizeddf['Plot'] = tokenizeddf['Plot'].apply(words2ints)

print(tokenizeddf.head())

# Pad the plot column with zeros to make all plots the same length
def pad_plot(plot):
    return plot + [0] * (maxlen - len(plot))

tokenizeddf['Plot'] = tokenizeddf['Plot'].apply(pad_plot)

                                    Title  \
0                             # (2012/II)   
1          #1 Cheerleader Camp (2010) (V)   
2                 #1 Serial Killer (2013)   
3  #1 at the Apocalypse Box Office (2015)   
4                             #137 (2011)   

                                                Plot  Action  Adventure  \
0  [73578, 197008, 406905, 259121, 73578, 77853, ...       0          0   
1  [1887, 116539, 61002, 415459, 234436, 120942, ...       0          0   
2  [284671, 443029, 510290, 278408, 29922, 73578,...       0          0   
3  [29684, 386494, 159351, 111924, 8116, 159351, ...       0          0   
4  [89540, 245013, 386494, 351170, 271014, 24025,...       0          0   

   Animation  Biography  Comedy  Crime  Drama  Family  ...  Horror  Music  \
0          1          0       0      0      0       0  ...       0      0   
1          0          0       1      0      0       0  ...       0      0   
2          0          0       0      0      0  

In [8]:
# Make a dataframe with just title, plot, and action columns
# Going to try this out first on just action movies
actiondf = tokenizeddf[['Title', 'Plot', 'Action']]
print(actiondf.head())

# Encode the action column with LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
actiondf["Action"] = le.fit_transform(actiondf["Action"])

                                    Title  \
0                             # (2012/II)   
1          #1 Cheerleader Camp (2010) (V)   
2                 #1 Serial Killer (2013)   
3  #1 at the Apocalypse Box Office (2015)   
4                             #137 (2011)   

                                                Plot  Action  
0  [73578, 197008, 406905, 259121, 73578, 77853, ...       0  
1  [1887, 116539, 61002, 415459, 234436, 120942, ...       0  
2  [284671, 443029, 510290, 278408, 29922, 73578,...       0  
3  [29684, 386494, 159351, 111924, 8116, 159351, ...       0  
4  [89540, 245013, 386494, 351170, 271014, 24025,...       0  


C:\Users\zkand\AppData\Local\Temp\ipykernel_9328\2112378440.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actiondf["Action"] = le.fit_transform(actiondf["Action"])


In [9]:
# Create a Dataset class for the actiondata
class GenreDataset(torch.utils.data.Dataset):
    def __init__(self, data, genre):
        self.plot = data["Plot"].values
        self.genre = data[genre].values

    def __len__(self):
        return len(self.plot)
    
    def __getitem__(self, i):
        plot = self.plot[i]
        genre = self.genre[i]
        return torch.tensor(plot, dtype=torch.long), torch.tensor(genre, dtype=torch.float)
    
# Train test split the actiondf
from sklearn.model_selection import train_test_split
actiontrain, actiontest = train_test_split(actiondf, test_size=0.2, random_state=42)
    
# Create a GenreDataset object for the actiondata
actiontraindataset = GenreDataset(actiontrain, "Action")
actiontestdataset = GenreDataset(actiontest, "Action")

# Create a DataLoader for the actiondataset
actiontrainloader = torch.utils.data.DataLoader(actiontraindataset, batch_size=32, shuffle=True)
actiontestloader = torch.utils.data.DataLoader(actiontestdataset, batch_size=32, shuffle=False)

In [10]:
# Create the RNN to classify the plots as action or not
class GenreRNN(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(GenreRNN, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), hidden).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])        
        return out

# Set the hyperparameters
embed = 128
hidden = 128
output = 2

# Create the model
actionmodel = GenreRNN(numwords, embed, hidden, output)

In [40]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(actionmodel.parameters(), lr=0.001)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Send the model to the device
actionmodel.to(device)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    epoch_loss = 0
    actionmodel.train()
    for plots, genres in actiontrainloader:
        # Send the data to the device
        plots = plots.to(device)
        genres = genres.to(device)
        outputs = actionmodel(plots)
        loss = criterion(outputs, genres.long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/5, Loss: 0.20157073438167572
Epoch 2/5, Loss: 0.4095592498779297
Epoch 3/5, Loss: 0.20661020278930664
Epoch 4/5, Loss: 0.587815523147583
Epoch 5/5, Loss: 0.19147908687591553


In [41]:
# Evaluate the model
actionmodel.eval()
correct = 0
total = 0
# Send the model to the device
actionmodel.to(device)
# Don't calculate gradients
with torch.no_grad():
    for plots, genres in actiontestloader:
        # Send the data to the device
        plots = plots.to(device)
        genres = genres.to(device)
        outputs = actionmodel(plots)
        _, predicted = torch.max(outputs.data, 1)
        total += genres.size(0)
        num_correct = (predicted == genres).sum().item()
        correct += num_correct
        print(correct, total)
accuracy = correct / total
print(f"Accuracy: {accuracy}")

28 32
59 64
88 96
116 128
145 160
172 192
203 224
233 256
261 288
288 320
317 352
348 384
376 416
408 448
436 480
464 512
491 544
519 576
548 608
580 640
609 672
640 704
668 736
700 768
729 800
760 832
790 864
818 896
845 928
875 960
907 992
938 1024
964 1056
994 1088
1024 1120
1051 1152
1079 1184
1110 1216
1140 1248
1170 1280
1199 1312
1229 1344
1256 1376
1285 1408
1315 1440
1344 1472
1374 1504
1404 1536
1434 1568
1463 1600
1492 1632
1522 1664
1550 1696
1579 1728
1611 1760
1639 1792
1666 1824
1694 1856
1724 1888
1754 1920
1786 1952
1815 1984
1843 2016
1873 2048
1904 2080
1934 2112
1965 2144
1995 2176
2019 2208
2048 2240
2077 2272
2107 2304
2139 2336
2168 2368
2199 2400
2228 2432
2257 2464
2289 2496
2320 2528
2347 2560
2377 2592
2406 2624
2434 2656
2464 2688
2490 2720
2516 2752
2544 2784
2570 2816
2601 2848
2633 2880
2660 2912
2687 2944
2717 2976
2744 3008
2772 3040
2801 3072
2832 3104
2862 3136
2892 3168
2920 3200
2949 3232
2977 3264
3006 3296
3037 3328
3062 3360
3092 3392
3120 3424
3

In [44]:
# Save the model
# Create the models directory if it doesn't exist
import os
if not os.path.exists("models/rnn"):
    os.makedirs("models/rnn")
torch.save(actionmodel.state_dict(), "models/rnn/actionmodel.pth")

From here, we'll train an RNN for each genre.

In [16]:
# Function to train a model
def train_genre_rnn(genre, epochs, embed, hidden, output):
    genremodel = GenreRNN(numwords, embed, hidden, output)
    # Extract the relevant columns from the dataframe
    genredf = tokenizeddf[['Title', 'Plot', genre]]
    # Encode the genre column
    genredf[genre] = le.fit_transform(genredf[genre])
    # Train test split the data
    # We won't use the test data in this function
    genretrain, genretest = train_test_split(genredf, test_size=0.2, random_state=42)
    # Create a GenreDataset object
    genretraindataset = GenreDataset(genretrain, genre)
    # Create a DataLoader
    genretrainloader = torch.utils.data.DataLoader(genretraindataset, batch_size=32, shuffle=True)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(actionmodel.parameters(), lr=0.001)

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Send the model to the device
    genremodel.to(device)

    # Train the model
    num_epochs = epochs
    for epoch in range(num_epochs):
        epoch_loss = 0
        genremodel.train()
        for plots, genres in genretrainloader:
            # Send the data to the device
            plots = plots.to(device)
            genres = genres.to(device)
            outputs = genremodel(plots)
            loss = criterion(outputs, genres.long())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # Return the model
    return genremodel

In [12]:
# Save a model
def save_model(model, genre):
    if not os.path.exists("models/rnn"):
        os.makedirs("models/rnn")
    torch.save(model.state_dict(), f"models/rnn/{genre}model.pth")

In [13]:
# Get all the genres
genres = data.columns[2:]
print(genres)

Index(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western'],
      dtype='object')


In [17]:
# For each genre, check if there's already a model
# If not, train a model and save it
for genre in genres:
    if not os.path.exists(f"models/rnn/{genre}model.pth"):
        print(f"Training model for {genre}")
        model = train_genre_rnn(genre, 5, 128, 128, 2)
        save_model(model, genre)
    else:
        print(f"Model for {genre} already exists")

Training model for Action


C:\Users\zkand\AppData\Local\Temp\ipykernel_9328\1652754105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/5, Loss: 0.5859911441802979
Epoch 2/5, Loss: 0.5859912633895874
Epoch 3/5, Loss: 0.6159600019454956
Epoch 4/5, Loss: 0.5660120248794556
Epoch 5/5, Loss: 0.5959808826446533
Training model for Adventure


C:\Users\zkand\AppData\Local\Temp\ipykernel_9328\1652754105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/5, Loss: 0.7564619183540344
Epoch 2/5, Loss: 0.7471478581428528
Epoch 3/5, Loss: 0.7564618587493896
Epoch 4/5, Loss: 0.7657760381698608
Epoch 5/5, Loss: 0.7704330086708069
Training model for Animation


...

Epoch 1/5, Loss: 0.7093785405158997
Epoch 2/5, Loss: 0.7056778073310852
Epoch 3/5, Loss: 0.7093785405158997
Epoch 4/5, Loss: 0.7106121182441711
Epoch 5/5, Loss: 0.7093785405158997
Training model for War


C:\Users\zkand\AppData\Local\Temp\ipykernel_9328\1652754105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/5, Loss: 0.7915686964988708
Epoch 2/5, Loss: 0.8048267960548401
Epoch 3/5, Loss: 0.7915686368942261
Epoch 4/5, Loss: 0.7981977462768555
Epoch 5/5, Loss: 0.7849395275115967
Training model for Western


C:\Users\zkand\AppData\Local\Temp\ipykernel_9328\1652754105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/5, Loss: 0.626314103603363
Epoch 2/5, Loss: 0.626314103603363
Epoch 3/5, Loss: 0.6306408047676086
Epoch 4/5, Loss: 0.626314103603363
Epoch 5/5, Loss: 0.6263141632080078


Old stuff below here

In [263]:
# Make a dataframe that has only the title, plot, and action columns
actiondata = data[["Title", "Plot", "Action"]]
# Get the first 5 rows of the actiondata
print(actiondata.head())

# Encode the action column with LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
actiondata["Action"] = le.fit_transform(actiondata["Action"])

# Get the length of the longest plot (for padding later on)
maxlen = actiondata["Plot"].apply(len).max()

# Get the set of all words in the plot column
wordset = set()
for plot in actiondata["Plot"]:
    wordset.update(plot)
# Get the number of unique words
numwords = len(wordset)
print("Number of unique words:", numwords)

# Create a dictionary that maps words to integers
word2int = {word: i for i, word in enumerate(wordset)}

# Function to convert a list of words to a list of integers
def words2ints(words):
    return [word2int[word] for word in words]

# Convert the plot column to a list of integers
actiondata["Plot"] = actiondata["Plot"].apply(words2ints)

# Get the first 5 rows of the actiondata
print(actiondata.head())

# Pad the plot column with zeros to make all plots the same length
def pad_plot(plot):
    return plot + [0] * (maxlen - len(plot))

actiondata["Plot"] = actiondata["Plot"].apply(pad_plot)

# Get the first 5 rows of the actiondata
print(actiondata.head())

KeyboardInterrupt: 

In [ ]:
# Create a Dataset class for the actiondata
class GenreDataset(torch.utils.data.Dataset):
    def __init__(self, data, genre):
        self.plot = data["Plot"].values
        self.genre = data[genre].values

    def __len__(self):
        return len(self.plot)
    
    def __getitem__(self, i):
        plot = self.plot[i]
        genre = self.genre[i]
        return torch.tensor(plot, dtype=torch.long), torch.tensor(genre, dtype=torch.float)
    
# Create a GenreDataset object for the actiondata
actiondataset = GenreDataset(actiondata, "Action")

# Create a DataLoader for the actiondataset
actionloader = torch.utils.data.DataLoader(actiondataset, batch_size=32, shuffle=True)

In [ ]:
# Create the RNN to classify the plots as action or not
class GenreRNN(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(GenreRNN, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), hidden).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])        
        return out

# Set the hyperparameters
embed = 128
hidden = 128
output = 2

# Create the model
actionmodel = GenreRNN(numwords, embed, hidden, output)

print(actiondataset[0])

(tensor([10155, 15944, 14626,  ...,     0,     0,     0]), tensor(0.))


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(actionmodel.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 0
    actionmodel.train()
    for plots, genres in actionloader:
        outputs = actionmodel(plots)
        loss = criterion(outputs, genres.long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 0.38106486201286316
Epoch 2/10, Loss: 0.13124583661556244
Epoch 3/10, Loss: 0.08345924317836761
Epoch 4/10, Loss: 0.09120485931634903
Epoch 5/10, Loss: 0.6385515332221985
Epoch 6/10, Loss: 0.3833125829696655
Epoch 7/10, Loss: 0.07289722561836243
Epoch 8/10, Loss: 0.12186636030673981
Epoch 9/10, Loss: 0.3818072974681854
Epoch 10/10, Loss: 0.38340580463409424


In [ ]:
# Look at the first five plots and their predicted genres
actionmodel.eval()
with torch.no_grad():
    for plots, genres in actionloader:
        outputs = actionmodel(plots)
        _, preds = torch.max(outputs.data, 1)
        #print("Predictions:", preds)
        #print("Actual:", genres)
        print((preds == genres)[0])
        print((preds == genres)[0].sum())
        print((preds == genres)[0].sum().item())
        num_correct = (preds == genres).sum().item()
        print(num_correct, "correct out of", len(genres))
        break

tensor(True)
tensor(1)
1
29 correct out of 32


In [ ]:
# Evaluate the model
actionmodel.eval()
correct = 0
total = 0
with torch.no_grad():
    for plots, genres in actionloader:
        outputs = actionmodel(plots)
        _, predicted = torch.max(outputs.data, 1)
        total += genres.size(0)
        num_correct = (predicted == genres).sum().item()
        correct += num_correct
print(correct, total)
accuracy = correct / total
print(f"Accuracy: {accuracy}")

916 1000
Accuracy: 0.916
